In [1]:
!pip install torch torchvision Pillow numpy matplotlib - -quiet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

In [3]:
# ----------------------------
# 1. Архитектура модели
# ----------------------------
class SimpleAntiAliasNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [4]:
# ----------------------------
# 2. Основные функции
# ----------------------------
def process_image(image_path, model_path=None, show_result=True):
    """Обработка одного изображения"""
    # Устройство (GPU/CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка модели
    model = SimpleAntiAliasNet().to(device)
    if model_path:
        model.load_state_dict(torch.load(model_path, map_location=device))

    model.eval()

    # Загрузка и преобразование изображения
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    input_tensor = transform(image).unsqueeze(0).to(device)

    # Обработка
    with torch.no_grad():
        output = model(input_tensor)

    # Преобразование результата
    output = output.squeeze(0).permute(1, 2, 0).cpu().numpy()
    output = np.clip(output, 0, 1)
    result = (output * 255).astype(np.uint8)
    result_image = Image.fromarray(result)

    # Сохранение и отображение
    result_image.save('result.png')

    if show_result:
        plt.figure(figsize=(15, 6))

        plt.subplot(1, 2, 1)
        plt.title("Исходное изображение")
        plt.imshow(image)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("После обработки")
        plt.imshow(result_image)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    return result_image


def train_simple_model(input_path, target_path):
    """Обучение модели на одной паре изображений"""
    # Устройство
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка изображений
    input_img = Image.open(input_path).convert('RGB')
    target_img = Image.open(target_path).convert('RGB')

    # Преобразования
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    input_tensor = transform(input_img).unsqueeze(0).to(device)
    target_tensor = transform(target_img).unsqueeze(0).to(device)

    # Модель и оптимизация
    model = SimpleAntiAliasNet().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Процесс обучения
    losses = []
    print("Начало обучения...")
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(input_tensor)
        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if (epoch + 1) % 10 == 0:
            print(f"Эпоха {epoch + 1}/100, Loss: {loss.item():.4f}")

    # Сохранение модели
    torch.save(model.state_dict(), 'antialiasing_model.pth')
    print("Модель сохранена как 'antialiasing_model.pth'")

    # График обучения
    plt.plot(losses)
    plt.title("График обучения")
    plt.xlabel("Эпоха")
    plt.ylabel("Loss")
    plt.show()

    return model

In [5]:
# ----------------------------
# 3. Главный блок
# ----------------------------

print("1. Загрузите ИСХОДНОЕ изображение (с алиасингом)")
uploaded_input = files.upload()
input_name = list(uploaded_input.keys())[0]

print("\n2. Загрузите ЦЕЛЕВОЕ изображение (без алиасинга)")
uploaded_target = files.upload()
target_name = list(uploaded_target.keys())[0]

print("\n3. Обучение модели...")
train_simple_model(input_name, target_name)

print("\n4. Обработка исходного изображения обученной моделью")
process_image(input_name, model_path='antialiasing_model.pth')

print("\n5. Готово! Результат сохранен как 'result.png'")
files.download('result.png')
files.download('antialiasing_model.pth')

# Загрузка файла
weights = torch.load('antialiasing_model.pth')

# Просмотр структуры
print("Ключи в файле:", weights.keys())

# Просмотр весов первого слоя
print("Веса первого слоя:", weights['layers.0.weight'])

1. Загрузите ИСХОДНОЕ изображение (с алиасингом)


IndexError: list index out of range